In [33]:
import requests
from bs4 import BeautifulSoup
from lxml import html
from lxml import etree
import pandas as pd
import time
import urllib3

In [34]:
# Suppress InsecureRequestWarning when SSL verification is disabled
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# List of district URLs to scrape for completed transactions
district_urls = [
# 'https://sh.lianjia.com/chengjiao/jingan/',
# 'https://sh.lianjia.com/chengjiao/xuhui/',
# 'https://sh.lianjia.com/chengjiao/huangpu/',
# 'https://sh.lianjia.com/chengjiao/changning/',
# 'https://sh.lianjia.com/chengjiao/putuo/',
# 'https://sh.lianjia.com/chengjiao/pudong/',
# 'https://sh.lianjia.com/chengjiao/baoshan/',
# 'https://sh.lianjia.com/chengjiao/hongkou/',
'https://sh.lianjia.com/chengjiao/yangpu/',
# 'https://sh.lianjia.com/chengjiao/minhang/',
# 'https://sh.lianjia.com/chengjiao/jinshan/',
# 'https://sh.lianjia.com/chengjiao/jiading/',
# 'https://sh.lianjia.com/chengjiao/chongming/',
# 'https://sh.lianjia.com/chengjiao/fengxian/',
# 'https://sh.lianjia.com/chengjiao/songjiang/',
# 'https://sh.lianjia.com/chengjiao/qingpu/'
]

In [35]:
district_names = {
    'jingan': 'Jing\'an',
    'xuhui': 'Xuhui',
    'huangpu': 'Huangpu',
    'changning': 'Changning',
    'putuo': 'Putuo',
    'pudong': 'Pudong',
    'baoshan': 'Baoshan',
    'hongkou': 'Hongkou',
    'yangpu': 'Yangpu',
    'minhang': 'Minhang',
    'jinshan': 'Jinshan',
    'jiading': 'Jiading',
    'chongming': 'Chongming',
    'fengxian': 'Fengxian',
    'songjiang': 'Songjiang',
    'qingpu': 'Qingpu'
}

In [36]:
# Function to get the total number of houses for a given URL
# def get_total_houses(tree):
#     total_houses_xpath = "//div[@class='total fl']/span/text()"
#     total_houses = tree.xpath(total_houses_xpath)
#     return int(total_houses[0].strip()) if total_houses else 0
def get_total_houses(url):
    # Headers to simulate a request from a web browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cookie':'lianjia_uuid=4b8ebecf-ba0d-4659-8af2-038ede8d290d; crosSdkDT2019DeviceId=-7yow6p-qg82dj-xegrp6bmzvytet0-dk1aofnao; ftkrc_=90e33cb4-234f-406a-9f64-374f3b6b3613; lfrc_=a7259049-1cf7-46f9-a05d-33507cb90914; Qs_lvt_200116=1717815310; _smt_uid=6663c80e.301fb184; _ga=GA1.2.1330354299.1717815312; Qs_pv_200116=273629849449905730%2C3264493958862233600%2C1115181869123250200%2C50628163928605670%2C2851292200084200000; _ga_E91JCCJY3Z=GS1.2.1717815312.1.1.1717815345.0.0.0; _ga_MFYNHLJT0H=GS1.2.1717815312.1.1.1717815345.0.0.0; _ga_W9S66SNGYB=GS1.2.1718091886.1.0.1718091886.0.0.0; _ga_1W6P4PWXJV=GS1.2.1718091886.1.0.1718091886.0.0.0; _ga_WLZSQZX7DE=GS1.2.1718097254.2.0.1718097254.0.0.0; _ga_TJZVFLS7KV=GS1.2.1718097254.2.0.1718097254.0.0.0; _ga_4JBJY7Y7MX=GS1.2.1718097264.1.1.1718097271.0.0.0; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%2218ff5c57a83a5f-0174151e67b42f-1a525637-1484784-18ff5c57a8423d2%22%2C%22%24device_id%22%3A%2218ff5c57a83a5f-0174151e67b42f-1a525637-1484784-18ff5c57a8423d2%22%2C%22props%22%3A%7B%22%24latest_traffic_source_type%22%3A%22%E7%9B%B4%E6%8E%A5%E6%B5%81%E9%87%8F%22%2C%22%24latest_referrer%22%3A%22%22%2C%22%24latest_referrer_host%22%3A%22%22%2C%22%24latest_search_keyword%22%3A%22%E6%9C%AA%E5%8F%96%E5%88%B0%E5%80%BC_%E7%9B%B4%E6%8E%A5%E6%89%93%E5%BC%80%22%7D%7D; select_city=310000; lianjia_ssid=c4a18819-122e-4086-aef4-d976cbdf179c; Hm_lvt_9152f8221cb6243a53c83b956842be8a=1717815311,1718952721; _gid=GA1.2.22477221.1718952723; Hm_lvt_efa595b768cc9dc7d7f9823368e795f1=1718973171; Hm_lpvt_efa595b768cc9dc7d7f9823368e795f1=1718973171; login_ucid=2000000175055980; lianjia_token=2.001314cf0a73fe868a02b9e63b20473ca8; lianjia_token_secure=2.001314cf0a73fe868a02b9e63b20473ca8; security_ticket=fy8oAkXqypfp4Yah1if4IVJLw8JfKLaYcrrEZkwiJ6XclKA4aw6hcmh5oDXJ/P9jh6gMHxjeGbmuYQQxLuYIchZEP7uI+hMqelonDXF1JTFnkFqac8vA6hE8eRrmODVAg7VqkGwYeSLl5GrgRQ2TDbeuhxjw57hKzMJx9ncvAbg=; _gat=1; _gat_past=1; _gat_global=1; _gat_new_global=1; _gat_dianpu_agent=1; _ga_LRLL77SF11=GS1.2.1718979902.16.1.1718980102.0.0.0; _ga_GVYN2J1PCG=GS1.2.1718979902.16.1.1718980102.0.0.0; Hm_lpvt_9152f8221cb6243a53c83b956842be8a=1718980103' 
    }
    # Send the request
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the total number of houses element
        total_houses_element = soup.find("div", class_="total fl").find("span")
        if total_houses_element:
            total_houses = total_houses_element.text
            return total_houses  
        else:
            print("Not found")



# Function to add page numbers to district URLs
def generate_paginated_urls(url, total_houses, houses_per_page=30):
    total_pages = (int(total_houses) + int(houses_per_page) - 1) // int(houses_per_page)
    return [f"{url}pg{page}/" for page in range(1, total_pages + 1)]

# Function to parse a single page URL and extract house details
def parse_page(url, district_name):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cookie':'lianjia_uuid=4b8ebecf-ba0d-4659-8af2-038ede8d290d; crosSdkDT2019DeviceId=-7yow6p-qg82dj-xegrp6bmzvytet0-dk1aofnao; ftkrc_=90e33cb4-234f-406a-9f64-374f3b6b3613; lfrc_=a7259049-1cf7-46f9-a05d-33507cb90914; Qs_lvt_200116=1717815310; _smt_uid=6663c80e.301fb184; _ga=GA1.2.1330354299.1717815312; Qs_pv_200116=273629849449905730%2C3264493958862233600%2C1115181869123250200%2C50628163928605670%2C2851292200084200000; _ga_E91JCCJY3Z=GS1.2.1717815312.1.1.1717815345.0.0.0; _ga_MFYNHLJT0H=GS1.2.1717815312.1.1.1717815345.0.0.0; _ga_W9S66SNGYB=GS1.2.1718091886.1.0.1718091886.0.0.0; _ga_1W6P4PWXJV=GS1.2.1718091886.1.0.1718091886.0.0.0; _ga_WLZSQZX7DE=GS1.2.1718097254.2.0.1718097254.0.0.0; _ga_TJZVFLS7KV=GS1.2.1718097254.2.0.1718097254.0.0.0; _ga_4JBJY7Y7MX=GS1.2.1718097264.1.1.1718097271.0.0.0; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%2218ff5c57a83a5f-0174151e67b42f-1a525637-1484784-18ff5c57a8423d2%22%2C%22%24device_id%22%3A%2218ff5c57a83a5f-0174151e67b42f-1a525637-1484784-18ff5c57a8423d2%22%2C%22props%22%3A%7B%22%24latest_traffic_source_type%22%3A%22%E7%9B%B4%E6%8E%A5%E6%B5%81%E9%87%8F%22%2C%22%24latest_referrer%22%3A%22%22%2C%22%24latest_referrer_host%22%3A%22%22%2C%22%24latest_search_keyword%22%3A%22%E6%9C%AA%E5%8F%96%E5%88%B0%E5%80%BC_%E7%9B%B4%E6%8E%A5%E6%89%93%E5%BC%80%22%7D%7D; select_city=310000; lianjia_ssid=c4a18819-122e-4086-aef4-d976cbdf179c; Hm_lvt_9152f8221cb6243a53c83b956842be8a=1717815311,1718952721; _gid=GA1.2.22477221.1718952723; Hm_lvt_efa595b768cc9dc7d7f9823368e795f1=1718973171; Hm_lpvt_efa595b768cc9dc7d7f9823368e795f1=1718973171; login_ucid=2000000175055980; lianjia_token=2.001314cf0a73fe868a02b9e63b20473ca8; lianjia_token_secure=2.001314cf0a73fe868a02b9e63b20473ca8; security_ticket=fy8oAkXqypfp4Yah1if4IVJLw8JfKLaYcrrEZkwiJ6XclKA4aw6hcmh5oDXJ/P9jh6gMHxjeGbmuYQQxLuYIchZEP7uI+hMqelonDXF1JTFnkFqac8vA6hE8eRrmODVAg7VqkGwYeSLl5GrgRQ2TDbeuhxjw57hKzMJx9ncvAbg=; _gat=1; _gat_past=1; _gat_global=1; _gat_new_global=1; _gat_dianpu_agent=1; _ga_LRLL77SF11=GS1.2.1718979902.16.1.1718980102.0.0.0; _ga_GVYN2J1PCG=GS1.2.1718979902.16.1.1718980102.0.0.0; Hm_lpvt_9152f8221cb6243a53c83b956842be8a=1718980103' 
    }
    response = requests.get(url, headers=headers, verify=False)
    tree = etree.HTML(response.content)

    house_data = []
    house_elements = tree.xpath("//ul[@class='listContent']/li")
    for house in house_elements:
        data = {
            'district': district_name,
            'title': house.xpath(".//div[@class='title']/a/text()")[0].strip() if house.xpath(".//div[@class='title']/a/text()") else '',
            'link': house.xpath(".//div[@class='title']/a/@href")[0] if house.xpath(".//div[@class='title']/a/@href") else '',
            'orientation': house.xpath(".//div[@class='address']/div[@class='houseInfo']/text()")[0].strip() if house.xpath(".//div[@class='address']/div[@class='houseInfo']/text()") else '',
            'deal_date': house.xpath(".//div[@class='address']/div[@class='dealDate']/text()")[0] if house.xpath(".//div[@class='address']/div[@class='dealDate']/text()") else '',
            'total_price': house.xpath(".//div[@class='address']/div[@class='totalPrice']/span[@class='number']/text()")[0] + '万' if house.xpath(".//div[@class='address']/div[@class='totalPrice']/span[@class='number']/text()") else '',
            'position': house.xpath(".//div[@class='flood']/div[@class='positionInfo']/text()")[0].strip() if house.xpath(".//div[@class='flood']/div[@class='positionInfo']/text()") else '',
            'unit_price': house.xpath(".//div[@class='flood']/div[@class='unitPrice']/span[@class='number']/text()")[0] + '元/平' if house.xpath(".//div[@class='flood']/div[@class='unitPrice']/span[@class='number']/text()") else '',
            'features': ', '.join(house.xpath(".//div[@class='dealHouseInfo']/span[@class='dealHouseTxt']/span/text()")),
            'list_amount': house.xpath(".//div[@class='dealCycleeInfo']/span[@class='dealCycleTxt']/span[1]/text()")[0] if house.xpath(".//div[@class='dealCycleeInfo']/span[@class='dealCycleTxt']/span[1]/text()") else '',
            'transaction_day': house.xpath(".//div[@class='dealCycleeInfo']/span[@class='dealCycleTxt']/span[2]/text()")[0] if house.xpath(".//div[@class='dealCycleeInfo']/span[@class='dealCycleTxt']/span[2]/text()") else '',
            'agent_name': house.xpath(".//div[@class='agentInfoList']/a[@class='agent_name']/text()")[0] if house.xpath(".//div[@class='agentInfoList']/a[@class='agent_name']/text()") else '',
            'agent_link': house.xpath(".//div[@class='agentInfoList']/a[@class='agent_name']/@href")[0] if house.xpath(".//div[@class='agentInfoList']/a[@class='agent_name']/@href") else ''
        }
        house_data.append(data)
    return house_data

def fetch_district_areas(district_url):
    # Headers to simulate a request from a web browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cookie':'lianjia_uuid=4b8ebecf-ba0d-4659-8af2-038ede8d290d; crosSdkDT2019DeviceId=-7yow6p-qg82dj-xegrp6bmzvytet0-dk1aofnao; ftkrc_=90e33cb4-234f-406a-9f64-374f3b6b3613; lfrc_=a7259049-1cf7-46f9-a05d-33507cb90914; Qs_lvt_200116=1717815310; _smt_uid=6663c80e.301fb184; _ga=GA1.2.1330354299.1717815312; Qs_pv_200116=273629849449905730%2C3264493958862233600%2C1115181869123250200%2C50628163928605670%2C2851292200084200000; _ga_E91JCCJY3Z=GS1.2.1717815312.1.1.1717815345.0.0.0; _ga_MFYNHLJT0H=GS1.2.1717815312.1.1.1717815345.0.0.0; _ga_W9S66SNGYB=GS1.2.1718091886.1.0.1718091886.0.0.0; _ga_1W6P4PWXJV=GS1.2.1718091886.1.0.1718091886.0.0.0; _ga_WLZSQZX7DE=GS1.2.1718097254.2.0.1718097254.0.0.0; _ga_TJZVFLS7KV=GS1.2.1718097254.2.0.1718097254.0.0.0; _ga_4JBJY7Y7MX=GS1.2.1718097264.1.1.1718097271.0.0.0; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%2218ff5c57a83a5f-0174151e67b42f-1a525637-1484784-18ff5c57a8423d2%22%2C%22%24device_id%22%3A%2218ff5c57a83a5f-0174151e67b42f-1a525637-1484784-18ff5c57a8423d2%22%2C%22props%22%3A%7B%22%24latest_traffic_source_type%22%3A%22%E7%9B%B4%E6%8E%A5%E6%B5%81%E9%87%8F%22%2C%22%24latest_referrer%22%3A%22%22%2C%22%24latest_referrer_host%22%3A%22%22%2C%22%24latest_search_keyword%22%3A%22%E6%9C%AA%E5%8F%96%E5%88%B0%E5%80%BC_%E7%9B%B4%E6%8E%A5%E6%89%93%E5%BC%80%22%7D%7D; select_city=310000; lianjia_ssid=c4a18819-122e-4086-aef4-d976cbdf179c; Hm_lvt_9152f8221cb6243a53c83b956842be8a=1717815311,1718952721; _gid=GA1.2.22477221.1718952723; Hm_lvt_efa595b768cc9dc7d7f9823368e795f1=1718973171; Hm_lpvt_efa595b768cc9dc7d7f9823368e795f1=1718973171; login_ucid=2000000175055980; lianjia_token=2.001314cf0a73fe868a02b9e63b20473ca8; lianjia_token_secure=2.001314cf0a73fe868a02b9e63b20473ca8; security_ticket=fy8oAkXqypfp4Yah1if4IVJLw8JfKLaYcrrEZkwiJ6XclKA4aw6hcmh5oDXJ/P9jh6gMHxjeGbmuYQQxLuYIchZEP7uI+hMqelonDXF1JTFnkFqac8vA6hE8eRrmODVAg7VqkGwYeSLl5GrgRQ2TDbeuhxjw57hKzMJx9ncvAbg=; _gat=1; _gat_past=1; _gat_global=1; _gat_new_global=1; _gat_dianpu_agent=1; _ga_LRLL77SF11=GS1.2.1718979902.16.1.1718980102.0.0.0; _ga_GVYN2J1PCG=GS1.2.1718979902.16.1.1718980102.0.0.0; Hm_lpvt_9152f8221cb6243a53c83b956842be8a=1718980103' 
    }
    # Send the request
    response = requests.get(district_url, headers=headers)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the area where links are located
        area_div = soup.find('div', attrs={'data-role': 'ershoufang'})

        # Extract the areas and their respective links
        areas = area_div.find_all('a') if area_div else []
        area_data = {area.get_text(strip=True): area['href'] for area in areas if 'chengjiao' in area['href']}
        # # Display the extracted data
        # for area_name, link in area_data.items():
        #     print(f"{area_name}: {link}")
        return(list(area_data.values()))
    else:
        print("Failed to retrieve the webpage")

def fetch_number_of_room(area_url):
    number_of_room_list = ['l1/','l2/','l3/','l4/','l5/']
    url_number_of_room_list = [area_url + i for i in number_of_room_list]
    return list(zip(number_of_room_list,url_number_of_room_list))

# Main function to orchestrate scraping for each district
def scrape_districts():
    district_dataframes = {}
    for url in district_urls:
        district_key = url.split('/')[-2]
        district_name = district_names.get(district_key, 'Unknown')
        remove_list = ['/chengjiao/','/chengjiao/pudong/', '/chengjiao/minhang/', '/chengjiao/baoshan/', '/chengjiao/xuhui/', '/chengjiao/putuo/', '/chengjiao/yangpu/', '/chengjiao/changning/', '/chengjiao/songjiang/', '/chengjiao/jiading/', '/chengjiao/huangpu/', '/chengjiao/jingan/', '/chengjiao/hongkou/', '/chengjiao/qingpu/', '/chengjiao/fengxian/', '/chengjiao/jinshan/', '/chengjiao/chongming/']
        area_url_list = fetch_district_areas(url)
        area_url_list = list(set(area_url_list).difference(set(remove_list)))
        # print(area_url_list)
        all_data = []
        print(area_url_list)
        for area_url in area_url_list:
            area_url = url + area_url
            remove_str = "/" + district_key +"//chengjiao"
            area_url = area_url.replace(remove_str, "")
            area_key = area_url.split('/')[-2]
            # area_name = district_names.get(area_key, 'Unknown')
            print(f"Scraping district: {district_name}, Scraping area: {area_key}")
            # initial_response = requests.get(url, verify=False)
            # tree = etree.HTML(initial_response.content)
            total_houses = get_total_houses(area_url)
            if int(total_houses) <= 3000:
                print(f"Total houses in {district_name} district {area_key} area : {total_houses}")
                paginated_urls = generate_paginated_urls(area_url, total_houses)
                for page_url in paginated_urls:
                    all_data.extend(parse_page(page_url, district_name))
                    print(f"Scrapped pages: {page_url}, House count in this page:{len(parse_page(page_url, district_name))}")
                    # time.sleep(1)  # Increase sleep time to avoid being blocked
            else:
                area_room_list = fetch_number_of_room(area_url)
                for area_room,area_room_url in area_room_list:
                    area_room_total_houses = get_total_houses(area_room_url)
                    print(f"Total houses in {district_name} district {area_key} area {area_room} room: {area_room_total_houses}")
                    area_room_paginated_urls = generate_paginated_urls(area_room_url, area_room_total_houses)
                    for area_room_page_url in area_room_paginated_urls:
                        all_data.extend(parse_page(area_room_page_url, district_name))
                        print(f"Scrapped pages: {area_room_page_url}, House count in this page:{len(parse_page(area_room_page_url, district_name))}")
                        # time.sleep(1)  # Increase sleep time to avoid being blocked
                
        district_df = pd.DataFrame(all_data)
        district_dataframes[f'df_chengjiao_{district_key}'] = district_df
        district_df.to_csv(f"{district_key}_chengjiao_data.csv", index=False)
        print(f"Data for {district_name} saved to {district_key}_chengjiao_data.csv")
    return district_dataframes


In [37]:
district_dfs = scrape_districts()

['/chengjiao/huangxinggongyuan/', '/chengjiao/wujiaochang/', '/chengjiao/anshan/', '/chengjiao/xinjiangwancheng/', '/chengjiao/dongwaitan/', '/chengjiao/shanghaizhoubian/', '/chengjiao/zhongyuan1/', '/chengjiao/zhoujiazuilu/', '/chengjiao/kongjianglu/']
Scraping district: Yangpu, Scraping area: huangxinggongyuan
Total houses in Yangpu district huangxinggongyuan area l1/ room:  1961 
Scrapped pages: https://sh.lianjia.com/chengjiao/huangxinggongyuan/l1/pg1/, House count in this page:30
Scrapped pages: https://sh.lianjia.com/chengjiao/huangxinggongyuan/l1/pg2/, House count in this page:30
Scrapped pages: https://sh.lianjia.com/chengjiao/huangxinggongyuan/l1/pg3/, House count in this page:30
Scrapped pages: https://sh.lianjia.com/chengjiao/huangxinggongyuan/l1/pg4/, House count in this page:30
Scrapped pages: https://sh.lianjia.com/chengjiao/huangxinggongyuan/l1/pg5/, House count in this page:30
Scrapped pages: https://sh.lianjia.com/chengjiao/huangxinggongyuan/l1/pg6/, House count in thi